## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №2</center>
### <center>ФБ-32 Чупріна Вікторія </center>

Імпортуємо потрібні нам бібліотеки

In [1]:
import datetime
import requests
import urllib.request
import pandas as pd
import os


Скачуємо VHI індекси для всіх областей
Також додаємо в назву дату та час скачування

In [2]:
for ids in range(1, 28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"
    response = requests.get(url)
    if response.status_code == 200:
        if not os.path.exists('vhi'):
            os.mkdir('vhi')
            print(f'The folder is created')
        date_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        vhi_url = urllib.request.urlopen(url)
        file_name = f'vhi/vhi_id_{ids}_{date_now}.csv'
        out = open(file_name, 'wb')
        out.write(vhi_url.read())
        out.close()
        print(f"VHI from id {ids} was downloaded at {date_now}")
    else:
        print(f"Download complete")
        break

VHI from id 1 was downloaded at 2025-02-21_13-01-47
VHI from id 2 was downloaded at 2025-02-21_13-01-50
VHI from id 3 was downloaded at 2025-02-21_13-01-52
VHI from id 4 was downloaded at 2025-02-21_13-01-55
VHI from id 5 was downloaded at 2025-02-21_13-01-57
VHI from id 6 was downloaded at 2025-02-21_13-02-00
VHI from id 7 was downloaded at 2025-02-21_13-02-02
VHI from id 8 was downloaded at 2025-02-21_13-02-04
VHI from id 9 was downloaded at 2025-02-21_13-02-06
VHI from id 10 was downloaded at 2025-02-21_13-02-08
VHI from id 11 was downloaded at 2025-02-21_13-02-10
VHI from id 12 was downloaded at 2025-02-21_13-02-13
VHI from id 13 was downloaded at 2025-02-21_13-02-17
VHI from id 14 was downloaded at 2025-02-21_13-02-19
VHI from id 15 was downloaded at 2025-02-21_13-02-21
VHI from id 16 was downloaded at 2025-02-21_13-02-23
VHI from id 17 was downloaded at 2025-02-21_13-02-25
VHI from id 18 was downloaded at 2025-02-21_13-02-27
VHI from id 19 was downloaded at 2025-02-21_13-02-30
VH

Даємо імена стовбцям, зчитуємо файли у фрейм

In [6]:
folder = 'vhi'
files = os.listdir(folder)

df_all = []

for file_name in files:
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(f'{folder}/{file_name}', header=1, names=headers, skiprows=1)

    df = df.drop(columns=['empty'], errors='ignore')

    df['VHI'] = pd.to_numeric(df['VHI'], errors='coerce')
    df = df[df['VHI'] != -1].dropna()

    parts = file_name.split("_")
    if len(parts) > 2 and parts[2].isdigit():
        df['area'] = int(parts[2])
    else:
        print(f"Попередження: неможливо оприділити 'area' для файла {file_name}")
        continue 

    df_all.append(df)

df_all = pd.concat(df_all, ignore_index=True)

df_all = df_all.dropna(axis=1, how='all')

print(df_all)

Предупреждение: Невозможно определить 'area' для файла df_all.csv
        Year  Week    SMN     SMT    VCI    TCI    VHI  area
0       1982   2.0  0.063  261.53  55.89  38.20  47.04    10
1       1982   3.0  0.063  263.45  57.30  32.69  44.99    10
2       1982   4.0  0.061  265.10  53.96  28.62  41.29    10
3       1982   5.0  0.058  266.42  46.87  28.57  37.72    10
4       1982   6.0  0.056  267.47  39.55  30.27  34.91    10
...      ...   ...    ...     ...    ...    ...    ...   ...
176980  2024  48.0  0.135  278.17  55.23  11.86  33.55     9
176981  2024  49.0  0.133  277.08  57.71  10.86  34.29     9
176982  2024  50.0  0.130  276.49  59.45   8.68  34.07     9
176983  2024  51.0  0.128  276.45  62.53   5.55  34.04     9
176984  2024  52.0  0.129  276.48  66.13   3.71  34.92     9

[176985 rows x 8 columns]


Змінимо індекси областей

In [7]:
dict_areas = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12,
              16: 13, 17: 15, 18: 14, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
df_all["area"] = df_all["area"].replace(dict_areas)
df_all.to_csv(f'vhi/df_all.csv', index=False)

print(df_all)

        Year  Week    SMN     SMT    VCI    TCI    VHI  area
0       1982   2.0  0.063  261.53  55.89  38.20  47.04    21
1       1982   3.0  0.063  263.45  57.30  32.69  44.99    21
2       1982   4.0  0.061  265.10  53.96  28.62  41.29    21
3       1982   5.0  0.058  266.42  46.87  28.57  37.72    21
4       1982   6.0  0.056  267.47  39.55  30.27  34.91    21
...      ...   ...    ...     ...    ...    ...    ...   ...
176980  2024  48.0  0.135  278.17  55.23  11.86  33.55    20
176981  2024  49.0  0.133  277.08  57.71  10.86  34.29    20
176982  2024  50.0  0.130  276.49  59.45   8.68  34.07    20
176983  2024  51.0  0.128  276.45  62.53   5.55  34.04    20
176984  2024  52.0  0.129  276.48  66.13   3.71  34.92    20

[176985 rows x 8 columns]


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [8]:
def vhi(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI']

def vhi_min(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].min()

def vhi_max(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].max()

df_all['Year'] = df_all['Year'].astype(int)

In [9]:
vhi(6, 2005)

92916    47.42
92917    51.20
92918    53.68
92919    54.35
92920    54.66
         ...  
97332    54.48
97333    53.22
97334    50.36
97335    47.41
97336    45.59
Name: VHI, Length: 153, dtype: float64

In [10]:
vhi_min(6, 2005)


45.59

In [11]:
vhi_max(6, 2005)

66.13

Ряд VHI за вказаний діапазон років для вказаних областей;

In [12]:
def vhi_range(year_min, year_max, areas):
    if not isinstance(areas, list) or not areas:
        return print('Empty or not a list')
    return df_all[(df_all['Year'] >= year_min) & (df_all['Year'] <= year_max) & (df_all['area'].isin(areas))][['Year', 'VHI', 'area']]

In [13]:
vhi_range(2000, 2005, [5, 2, 3])

,Year,VHI,area
105785,2000,24.65,2
105786,2000,27.49,2
105787,2000,31.36,2
105788,2000,37.28,2
105789,2000,40.85,2
...,...,...,...
149772,2005,40.70,3
149773,2005,42.92,3
149774,2005,41.13,3
149775,2005,41.49,3


Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей

In [14]:
def extreme_droughts(procent):
    df_drouhts = df_all[(df_all['VHI'] <= 15) & (df_all['VHI'] != -1)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    result = grouped[grouped > (25 * procent / 100)].reset_index()
    return result

In [15]:
extreme_droughts(5)

,Year,area
0,1986,2
1,1993,2
2,2000,6
3,2007,5


Аналогічно для помірних посух

In [16]:
def moderate_droughts(procent, min=15, max=40):
    df_drouhts = df_all[(df_all['VHI'] >= min) & (df_all['VHI'] <= max)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    count_areas = df_all['area'].nunique()
    result = grouped[grouped > (count_areas * procent / 100)].reset_index()
    return result

In [17]:
moderate_droughts(20)

,Year,area
0,1982,25
1,1983,25
2,1984,25
3,1985,25
4,1986,25
5,1987,25
6,1988,25
7,1989,25
8,1990,25
9,1991,24
